# Resize Images from google images to 256x256

In [1]:
# import os 
# import numpy as np
# import cv2

# downloadsdir = './downloads'
# dirs = [x for x in os.walk(downloadsdir)]
# for shroom in dirs[0][1]:
#     resizeddir = './resized\\\\'+shroom
#     fullsizedir = './downloads\\\\'+shroom
#     if not os.path.exists(resizeddir):
#         os.makedirs(resizeddir)
#     pics = [x[2] for x in os.walk(fullsizedir)]
#     for idx, p in enumerate(pics[0]):
#         if not p.endswith('.gif'):
#             picpath = fullsizedir+'\\\\'+p
#             try:
#                 img = cv2.imread(picpath,-1)
#                 resized = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
#                 cv2.imwrite(resizeddir+'\\\\'+str(idx)+'.jpg', resized)
#             except:
#                 print('Could not process ' + p)


In [2]:
import pandas as pd
import numpy as np
import os
import cv2

num_classes = 16

X = []
y = []
# is_edible = []

# mushroom_info = pd.read_json('mushroom_classes.json', lines=True)
classdirs = [x for x in os.walk('./resized')]
for shroomidx, shroom in enumerate(classdirs[0][1]):
#     info = mushroom_info.loc[mushroom_info.name_latin == shroom]
#     edible = info.edibility.isin(("edible", "edible and good", "edible and excellent"))
#     is_edible.append(edible)
    imagepaths = [x for x in os.walk('./resized\\\\'+shroom)]
    for path in imagepaths[0][2]:
        img = cv2.imread('./resized\\\\'+shroom+'\\\\'+path,-1)
        if (img.shape == (224, 224, 3)):
            X.append(img)
            y.append(shroomidx)

X = np.stack(X)
y = np.stack(y)
# is_edible = pd.Series(is_edible)
print(X.shape)
print(y.shape)
# print(is_edible.shape)
    

(1010, 224, 224, 3)
(1010,)


In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator()

C:\Users\Kevin\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# Train and test data
import keras
# rescale !!
X = X/255.0

N = len(X)
N_tr = int(0.8*N)

# shuffle the data
indx = np.arange(N)
np.random.shuffle(indx)
X = X[indx]
y = y[indx]

# split
X_tr = X[0:N_tr]
y_tr = y[0:N_tr]
y_tr = keras.utils.to_categorical(y_tr, num_classes)

X_te = X[N_tr:]
y_te = y[N_tr:]
y_te = keras.utils.to_categorical(y_te, num_classes)


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

def my_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=X[0].shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='sigmoid'))
    return model



In [6]:
model = my_model()


from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau

# basemodel = VGG16(include_top=False,
#                   input_shape=(224,224,3))
# # for layer in basemodel.layers[1:13]:
# #     layer.trainable = False
    
# topmodel = Sequential()
# topmodel.add(Flatten(input_shape=basemodel.output_shape[1:]))
# topmodel.add(Dense(num_classes, activation='relu'))
# topmodel.add(Dropout(0.4))
# topmodel.add(Dense(num_classes, activation='sigmoid'))

# model = Model(inputs = basemodel.input, outputs = topmodel(basemodel.output))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])

model_json = model.to_json()
with open("models/model.json", "w") as json_file:
    json_file.write(model_json)


In [7]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 128)     36992     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 106, 106, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 51, 51, 256)       295168    
__________

In [ ]:
# fits the model on batches with real-time data augmentation:
N_BATCH = 16

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.85,
                              patience=5,
                              min_lr=0.001)
history = model.fit(X_tr, y_tr,
                    batch_size = N_BATCH,
                    epochs = 200,
                    verbose = 1,
                    validation_data = (X_te, y_te),
                    callbacks=[reduce_lr])

# save model weights and training history
model.save_weights('models/weights.h5')
import pickle
with open('models/history', 'wb') as f:
        pickle.dump(history.history, f)

Train on 808 samples, validate on 202 samples
Epoch 1/200
808/808 [==============================] - 16s 20ms/step - loss: 2.7723 - acc: 0.0656 - val_loss: 2.7728 - val_acc: 0.0396
Epoch 2/200
808/808 [==============================] - 12s 15ms/step - loss: 2.7710 - acc: 0.0903 - val_loss: 2.7735 - val_acc: 0.0446
Epoch 3/200
808/808 [==============================] - 12s 15ms/step - loss: 2.7707 - acc: 0.0780 - val_loss: 2.7740 - val_acc: 0.0446
Epoch 4/200
808/808 [==============================] - 12s 15ms/step - loss: 2.7705 - acc: 0.0705 - val_loss: 2.7740 - val_acc: 0.0446
Epoch 5/200
808/808 [==============================] - 12s 15ms/step - loss: 2.7690 - acc: 0.0817 - val_loss: 2.7746 - val_acc: 0.0446
Epoch 6/200
808/808 [==============================] - 12s 15ms/step - loss: 2.7695 - acc: 0.0569 - val_loss: 2.7746 - val_acc: 0.0446
Epoch 7/200
808/808 [==============================] - 12s 15ms/step - loss: 2.7667 - acc: 0.0755 - val_loss: 2.7749 - val_acc: 0.0446
Epoch 8/2

In [ ]:
# load the model training history
import matplotlib.pyplot as plt
with open("models/history", "rb") as f:
    hist = pickle.load(f)
    
plt.plot(hist["acc"], label = "train accuracy", color = "green")
plt.plot(hist["val_acc"], label = "test accuracy", color = "blue")
plt.legend()
plt.savefig("docs/training_history.png")
plt.show()

In [ ]:
model.save("./models/model1.h5")
# def predict_and_draw(i, X, y, model, classnames, save = False):
#     x = X[i]
#     x = x.astype(np.uint8)
#     plt.imshow(x)
#     x = x/ 255.0
#     x.shape = (1, ) + x.shape
#     p = model.predict(x)[0,0].round(2)
#     plt.title("P(edible): " + str(p) + " Actually edible: " + str(y[i]))
#     if(save):
#         plt.savefig("docs/prediction_"+str(i)+".png")
#     plt.show()
    
# draw_sample(1, X, y, classdirs[0][1])
